# Python Code

In [ ]:
import pandas as pd
from scipy.integrate import odeint
import numpy as np
import pylab as pl
import matplotlib.pyplot as plt
import random
from statistics import mean 
import itertools
import ga 

path_file="https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv"
dataset=pd.read_csv(path_file,parse_dates=['data'])

# guariti,infetti,sani, morti 
dataset['asintomatici']= dataset['totale_casi'] * 0.44 
dataset['immuni'] = dataset['dimessi_guariti'] * 0.05  #verificare proporzione
data=dataset[['totale_positivi','dimessi_guariti','deceduti','asintomatici','immuni']]

    
def set_coeff_randomly(minimo,massimo,n=9):
    return [random.uniform(minimo,massimo) for i in range(0,n)]

iniziale = set_coeff_randomly(0,1)
k1,k2,k3,k4,k5,k6,k7,k8,k9 =iniziale[0],iniziale[1],iniziale[2],iniziale[3],iniziale[4],iniziale[5],iniziale[6],iniziale[7],iniziale[8]

def perturbazione(ls):
    sup = 0.2
    for i in range(len(ls)):
        if ls[i] < 0.5:
            ls[i]=random.uniform(ls[i],ls[i]+sup)
        else:
            ls[i]=random.uniform(0,ls[i]-sup)
    return(ls)
 
def diffusione():
    orizzonte_temporale_in_gg=63
    Sani = 6036000
    Infetti=221
    Guariti=1
    Morti= 7
    Immuni = 8
    Asintomatici = 10
    y0 =  [Sani,Infetti,Morti,Guariti,Immuni,Asintomatici]
    t = np.linspace(0,orizzonte_temporale_in_gg,orizzonte_temporale_in_gg)
    ret = odeint(deriv, y0, t)
    return(ret)
    

def deriv(y,t):  
    '''set di equazioni differenziali'''
    Sani,Infetti,Asintomatici,Morti,Guariti,Immuni = y 
    dSanidt =  -k1*Sani*Infetti -k2*Sani*Infetti-k3*Sani*Asintomatici- k4*Sani*Asintomatici
    dInfettidt = k1*Sani*Infetti+k4*Sani-k6*Infetti-k7*Infetti
    dAsintomaticidt =k2*Sani*Infetti+k3*Sani*Asintomatici-k5*Asintomatici
    dMortidt=k6*Infetti
    dGuaritidt = k5*Asintomatici+k7*Infetti-k8*Guariti-k9*Guariti
    dImmunidt = k9*Guariti
    return dSanidt,dInfettidt,dAsintomaticidt,dMortidt,dGuaritidt,dImmunidt  

def fitness(previsti):
    df=pd.DataFrame(previsti,columns=['sani','totale_positivi','dimessi_guariti','deceduti','asintomatici','immuni'])
    df=df.drop(columns=['sani'])
    dataframe=((df-data)**2)
    lista=dataframe.values.tolist()
    merged = list(itertools.chain.from_iterable(lista))
    media=mean(merged)
    return(media) 
    

def localsearch(massimovaloreiniziale,iterazioni):
    iniziale = set_coeff_randomly(0,massimovaloreiniziale)
    k=[iniziale[0],iniziale[1],iniziale[2],iniziale[3],iniziale[4],iniziale[5],iniziale[6],iniziale[7],iniziale[8]]
    k1,k2,k3,k4,k5,k6,k7,k8,k9 =iniziale[0],iniziale[1],iniziale[2],iniziale[3],iniziale[4],iniziale[5],iniziale[6],iniziale[7],iniziale[8]
    errore=100000000000000000000000000000000000000
    val='1'
    for c in range(iterazioni):
        valori=diffusione()
        stimaerrore=fitness(valori)
        if stimaerrore<errore:
            errore=stimaerrore
            val=valori 
            k[0],k[1],k[2],k[3],k[4],k[5],k[6],k[7],k[8] =iniziale[0],iniziale[1],iniziale[2],iniziale[3],iniziale[4],iniziale[5],iniziale[6],iniziale[7],iniziale[8]
        iniziale = set_coeff_randomly(0,massimovaloreiniziale)  
        k1,k2,k3,k4,k5,k6,k7,k8,k9 =iniziale[0],iniziale[1],iniziale[2],iniziale[3],iniziale[4],iniziale[5],iniziale[6],iniziale[7],iniziale[8]
    return(k,val,errore)





### Iterated Local Search (ILS)

In [ ]:


def ils(iterazioni,max_iterazioni,massimovaloreiniziale):
    iniziale = set_coeff_randomly(0.000001,0.001)
    loc_search = localsearch(massimovaloreiniziale,iterazioni)
    k=loc_search[0]
    best_val = loc_search[1]
    best_errore = loc_search[2]
    #iter_errore = [best_errore]
    #iter_best_errore = [best_errore]
    for c in range(max_iterazioni):
        val=best_val
        nuova_iniziale = perturbazione(k)
        loc_search = localsearch(massimovaloreiniziale,iterazioni)
        val = loc_search[1]
        if loc_search[2] < best_errore: # Se nuovo errore minore best si aggiorna il best
            best_errore = loc_search[2]
            best_val= loc_search[1]
            k=loc_search[0]
       # elif loc_search[0] == best_pers and loc_search[1] < best_num: # se uguale si tiene come best il numero più piccolo
               # best_num =loc_search[1]
        #iter_pers.append(loc_search[0])
       # iter_best_pers.append(best_pers)
   # results = {"errore":iter_pers, "persistenza_migliore":iter_best_pers, "numero_migliore":best_num,"ripetizioni":conta(best_num)}
    return(k,best_val,best_errore)    
prova=ils(10,10,1)


def grafico(alpha,ret):
    fig = plt.figure(facecolor='w')
    ax = fig.add_subplot(111)
    t = np.linspace(0,61,63)
    ax.plot(t, ret[:,1], 'r', alpha=0.7 ,lw=2, label='Infetti')
    ax.plot(t, ret[:,2], 'g', alpha=0.7 ,lw=2, label='Deceduti')
    ax.plot(t, ret[:,3], 'k', alpha=0.7, lw=2, label='Guariti')
    ax.set_xlabel('tempo (in giorni)')
    ax.set_ylabel('Persone (x1000)')
    ax.set_ylim(0,700000)
    legend = ax.legend()
    plt.text(1,2800,'alpha='+str(alpha))
    for spine in ('top', 'right', 'bottom', 'left'):
        ax.spines[spine].set_visible(False)
    plt.show()
    
for alpha in np.linspace(0.1,1,10):
    grafico(alpha,prova[1])